# GCP PubSub Consumer

Simple script to consume and display messages from GCP PubSub for testing.

In [3]:
# Import required libraries
import os
import json
from google.cloud import pubsub_v1
from concurrent.futures import TimeoutError
import time

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/longnguyen/miniconda3/envs/spark_env/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/longnguyen/miniconda3/envs/spark_env/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.pubsub_v1 with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.pubsub_v1.
  warnings.warn(message, FutureWarning)


In [9]:
import dotenv
dotenv.load_dotenv('../.env')

True

In [10]:
# Set up GCP credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../credentials/gpc-key.json'

# Configuration
PROJECT_ID = os.getenv('PROJECT_ID')  # Replace with your GCP project ID
SUBSCRIPTION_ID = os.getenv('SUBSCRIPTION_ID')  # Replace with your subscription ID
subscription_path = f'projects/{PROJECT_ID}/subscriptions/{SUBSCRIPTION_ID}'

print(f"Subscription path: {subscription_path}")

Subscription path: projects/quickstart-1565940213534/subscriptions/local-test


In [11]:
# Simple function to pull and display messages
def pull_messages(max_messages=5):
    """Pull messages and display their format."""
    subscriber = pubsub_v1.SubscriberClient()
    
    response = subscriber.pull(
        request={"subscription": subscription_path, "max_messages": max_messages}
    )
    
    if not response.received_messages:
        print("No messages available")
        return
    
    print(f"Received {len(response.received_messages)} message(s)\n")
    
    ack_ids = []
    for i, received_message in enumerate(response.received_messages, 1):
        message = received_message.message
        
        print(f"--- Message {i} ---")
        print(f"ID: {message.message_id}")
        print(f"Published: {message.publish_time}")
        print(f"Data: {message.data.decode('utf-8')}")
        
        if message.attributes:
            print(f"Attributes: {dict(message.attributes)}")
        print()
        
        ack_ids.append(received_message.ack_id)
    
    # Acknowledge messages
    subscriber.acknowledge(
        request={"subscription": subscription_path, "ack_ids": ack_ids}
    )
    print(f"✓ Acknowledged {len(ack_ids)} message(s)")

In [12]:
# Pull and display messages
pull_messages(max_messages=5)

Received 3 message(s)

--- Message 1 ---
ID: 17443419501820294
Published: 2025-12-17 16:00:30.732000+00:00
Data: {"schema":{"type":"struct","fields":[{"type":"int32","optional":false,"field":"product_id"},{"type":"string","optional":false,"field":"product_name"},{"type":"int32","optional":false,"field":"aisle_id"},{"type":"int32","optional":false,"field":"department_id"},{"type":"string","optional":true,"field":"__deleted"},{"type":"string","optional":true,"field":"__op"},{"type":"int64","optional":true,"field":"__source_ts_ms"},{"type":"string","optional":true,"field":"__source_table"}],"optional":false,"name":"cdc-pipeline.public.products.Value"},"payload":{"product_id":1,"product_name":"Whole Milk","aisle_id":5,"department_id":1,"__deleted":"false","__op":"r","__source_ts_ms":1765987227873,"__source_table":"products"}}
Attributes: {'__op': '{"schema":{"type":"string","optional":true},"payload":"r"}'}

--- Message 2 ---
ID: 17441873313009624
Published: 2025-12-17 16:00:48.457000+00:0